Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.0 MB/s eta 0:00:00
  Created wheel for fse: filename=fse-1.0.0-cp310-cp310-linux_x86_64.whl size=301842 sha256=d0a28966645b8e83cc5de3ceb4eb7fa7699f92405c48a5395cafee456ce91cd6
  Stored in directory: /root/.cache/pip/wheels/4c/8a/53/df807a30bb717ec61730522005512e2dd2656e3eca7e5f60f7
Successfully built fse
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.3 MB/s eta 0:

In [ ]:
import math
import os
import re
import fse
import gensim
import gensim.downloader as api
import nltk
import numpy as np
import pandas as pd
from fse import SplitIndexedList
from fse.models import uSIF
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('stopwords')
import logging
import re
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from nltk.tokenize import word_tokenize
from transformers import BertForQuestionAnswering, BertTokenizer
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-3-beeaa5b37d76>:25: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
# Reading in the dataset
df1 = pd.read_csv('/content/S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('/content/S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('/content/S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')
frames = [df1, df2, df3]
df = pd.concat(frames)

FileNotFoundError: ignored

In [ ]:
def getArticleText(file):
  fpath = './dataset/text_data/'+file+'.txt.clean'
  try:
    f = open(fpath, 'r')
    text = f.read()
  except UnicodeDecodeError:
    f = open(fpath, 'r', encoding = 'ISO-8859-1')
    text = f.read()
  return text

In [ ]:
df = df.dropna(subset=['ArticleFile'])
df = df.dropna(subset=['Answer'])
df['ArticleText'] = df['ArticleFile'].apply(lambda x: getArticleText(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: re.sub(r'(\n)+', '. ', x))
df = df.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis='columns')
df

In [ ]:
# stop_words = set(stopwords.words('english'))
def cleanQuestion(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanAnswer(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanText(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s\.\?]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

In [ ]:
df['Question'] = df['Question'].apply(lambda x: cleanQuestion(x))
df['Answer'] = df['Answer'].apply(lambda x: cleanAnswer(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: cleanText(x))
df

In [ ]:
# Preparing data for training
allQuestion = '. '.join(list(df['Question']))
allAnswer = '. '.join(list(df['Answer']))
allContext = '. '.join(list(df['ArticleText']))
text = allQuestion + allAnswer + allContext
s = SplitIndexedList(text.split('.'))

## **Word2Vec**

In [ ]:
dataset = []
title = ""
for i in range(0, len(df), 2):
    this_title = df.iloc[i]['ArticleTitle']
    if (this_title!=title):
        title = this_title
        text = df.iloc[i]['ArticleText']
        splitted = text.split(sep='.')
        for j in range(len(splitted)):
            text = splitted[j]
            if(text!=''):
                words = text.split()
                dataset.append(words)
    dataset.append(df.iloc[i]['Question'].split())
    dataset.append(df.iloc[i]['Answer'].split())

In [ ]:
model_wv = gensim.models.Word2Vec(dataset, size=100, window=8, min_count=1, sg=0, workers=8) # I have 8 cpu cores
# sg = {0, 1} – Training algorithm: 1 for skip-gram; otherwise CBOW

In [ ]:
model_wv.train(dataset, total_examples=len(dataset), compute_loss=True, epochs=50)

In [ ]:
def get_embedding(sentence):
  pos_sum = [0.0 for i in range(100)]
  num = 0
  words = sentence.split()
  for i in words:
    try:
      embed = model_wv.wv[i]
    except:
      continue
    else:
      pos_sum += embed
      num +=1
  if(num==0):
    return pos_sum
  else:
    pos_sum /= num
    return pos_sum

# get answer using euclidean distance
def get_answer(question, answer_para):
  question_embedding = get_embedding(rem_stop(question))
  min_distance = math.inf
  answer = 0
  for i in range(len(answer_para)):
    answer_embedding = get_embedding(rem_stop(answer_para[i]))
    distance = np.linalg.norm(question_embedding-answer_embedding)
    if (distance < min_distance):
      answer = i
      # print(answer)
      min_distance = distance
  return answer_para[answer]

def rem_stop(sentence):
    strr=''
    my_string = sentence.split()
    for i in range(len(my_string)):
        if my_string[i] not in stopwords.words('english'):
            strr = strr+' '+my_string[i]
    return strr[1:]

# get answer using cosine similarity
def get_answer_cosine(question, answer_para):
  question_embedding = get_embedding(rem_stop(question))
  max_similarity = -math.inf
  answer = 0
  for i in range(len(answer_para)):
    answer_embedding = get_embedding(rem_stop(answer_para[i]))
    similarity = cosine_similarity(np.expand_dims(question_embedding,0), np.expand_dims(answer_embedding,0))
    if (similarity > max_similarity):
      answer = i
      max_similarity = similarity
  return answer_para[answer]

In [ ]:
index = 296
my_text = df.iloc[index]['ArticleText']
temp_sentences = my_text.split(sep='.')
sentences=[]
for i in range(len(temp_sentences)):
    if(temp_sentences[i]!=''):
        sentences.append(temp_sentences[i])
my_question = df.iloc[index]['Question']

# Function to break up article text into individual sentences
def contextToSents(my_text):
  temp_sentences = my_text.split(sep='.')
  sentences=[]
  for i in range(len(temp_sentences)):
      if(temp_sentences[i]!=''):
          sentences.append(temp_sentences[i])
  return sentences

In [ ]:
# Word2Vec outputs
print(my_question) # Actual Question
print(rem_stop(my_question)) # Answer without stopwords
print(df.iloc[index]['Answer']) # Actual Answer
print(get_answer(my_question, sentences)) # Our model's prediction using euclidean distance
print("\n")
print(get_answer_cosine(my_question, sentences)) # Our model's prediction using cosine similarity

## **SIF Embedding**

In [ ]:
from fse.models import uSIF
glove = api.load("glove-wiki-gigaword-100")
model_sif = uSIF(glove, workers=2, lang_freq="en")

In [ ]:
model_sif.train(s)

In [ ]:
q = 'was abraham lincoln the sixteenth president of the united states'
c1 = 'Abraham Lincoln (February 12, 1809 – April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination.'
c2 = 'Scholars now rank Lincoln among the top three U.S. Presidents, with the majority of those surveyed placing him first.'
c3 = 'Symbolic log cabin at Abraham Lincoln Birthplace National Historic Site.'

tmpq = (q.split(), 0)
tmpc1 = (c1.split(), 0)
tmpc2 = (c2.split(), 0)
tmpc3 = (c3.split(), 0)

print(metrics.pairwise.cosine_similarity(model_sif.infer([tmpq]), model_sif.infer([tmpc1])))
print(metrics.pairwise.cosine_similarity(model_sif.infer([tmpq]), model_sif.infer([tmpc2])))
print(metrics.pairwise.cosine_similarity(model_sif.infer([tmpq]), model_sif.infer([tmpc3])))

In [ ]:
def getSim(q, x):
  x = (str(x).split(), 0)
  sim = metrics.pairwise.cosine_similarity(model_sif.infer([q]), model_sif.infer([x]))
  return sim

def getAnswer(question, context):
  q = (str(question).split(), 0)
  c = pd.DataFrame(str(context).split('.'))
  c['sim'] = c[0].apply(lambda x: getSim(q, x))
  max = c.sort_values(by='sim', ascending=False).iloc[:3]
  return max

def getBestAnswer(question, potentials):
  q = (str(question).split(), 0)
  c = pd.DataFrame(potentials)
  c['sim'] = c[0].apply(lambda x: getSim(q, x))
  max = c.sort_values(by='sim', ascending=False).iloc[:3]
  return max[0]

In [ ]:
qno = 250
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
print(getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno]))

In [ ]:
qno = 67
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
sol = getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno])
print(sol)

## **BERT with SIF**

In [ ]:
use_cuda = True
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
def getGraph(start_scores, end_scores, tokens):

  # x-axis: unique tokens
  # y-axis: start word score for each token

  # Pull the scores from Tensors and convert to 1-D np arrays
  start_scores = start_scores.detach().numpy().flatten()
  end_scores = end_scores.detach().numpy().flatten()

  # We add unique token index to each token
  token_label = list()
  for (i, t) in enumerate(tokens):
      token_label.append('{:} - {:>2}'.format(t, i))

  # Plot Graph
  sns.set(style='darkgrid')
  plt.rcParams["figure.figsize"] = (24,8)
  ax = sns.barplot(x=token_labels, y=s_scores, ci=None)
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
  ax.grid(True)
  plt.title('Start Word Scores for Each unique Token in Context')
  plt.show()

In [ ]:
model_sif.train(SplitIndexedList(list(df['ArticleText'])))

In [ ]:
def get_split(text1):

  #Reference: https://medium.com/@armandj.olivares/using-bert-for-classifying-documents-with-long-texts-5c3e7b04573d

  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else:
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:250]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 250]
      l_total.append(" ".join(l_parcial))
  return l_total

In [ ]:
def getAnswerBert(question, context):

  # print('Query Context has {} tokens.'.format(len(tokenizer.encode(context))))

  context_list = get_split(context)

  ans = []

  for c in context_list:

    encoding = tokenizer.encode_plus(text=question,text_pair=c)

    inputs = encoding['input_ids']  #Token embeddings
    token_type_id = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

    output = model_bert(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([token_type_id]))
    start_index = torch.argmax(output.start_logits)
    end_index = torch.argmax(output.end_logits)

    answer = ' '.join(tokens[start_index:end_index+1])

    ans.append(answer)
  print('Question: ', question)

  potentials = []
  for i in ans:
    if ('SEP' not in i) and ('CLS' not in i):
      potentials.append(re.sub('(#)+', '', i))

  answer = getBestAnswer(question, potentials)

  # print('Potential Answers: \n')
  # print(answer.head())
  return answer

In [ ]:
print(getAnswerBert(df['Question'].iloc[171], df['ArticleText'].iloc[171]))
print('Actual Answer: ', df['Answer'].iloc[171])

In [ ]:
print(getAnswerBert(df['Question'].iloc[255], df['ArticleText'].iloc[255]))
print('Actual Answer: ', df['Answer'].iloc[255])

## **Comparision**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Function to compare all 3 of our models

def compare(indexes):
  wv_sim = []
  sif_sim = []
  bert_sim = []
  for i in indexes:
    question = df.iloc[i]['Question']
    context = df.iloc[i]['ArticleText']
    answer = df.iloc[i]['Answer']

    if ('yes' in answer.lower()) or ('no' in answer.lower()):
      continue

    ans_sif = model_sif.infer([((answer).split(), 0)])

    w = metrics.pairwise.cosine_similarity(model_sif.infer([((get_answer_cosine(question, contextToSents(context))).split(), 0)]), ans_sif)
    s = metrics.pairwise.cosine_similarity(model_sif.infer([((getAnswer(question, context).iloc[0][0]).split(), 0)]), ans_sif)

    b_ans = getAnswerBert(question, context)
    b = metrics.pairwise.cosine_similarity(model_sif.infer([((b_ans[b_ans.keys()[0]]).split(), 0)]), ans_sif)

    wv_sim.append(w)
    sif_sim.append(s)
    bert_sim.append(b)

  print('WV:', wv_sim)
  print('SIF:', sif_sim)
  print('BERT:', bert_sim)

  return wv_sim, sif_sim, bert_sim

In [ ]:
# Function to compare all 3 of our models visually
def visualise(i):

  question = df.iloc[i]['Question']
  context = df.iloc[i]['ArticleText']
  answer = df.iloc[i]['Answer']

  w_ans = get_answer_cosine(question, contextToSents(context))
  s_ans = getAnswer(question, context).iloc[0][0]
  b_ans = getAnswerBert(question, context)

  print('Question: ', question)
  print('Real: ', answer)
  print('WV: ', w_ans)
  print('SIF: ', s_ans)
  print('BERT: ', b_ans[b_ans.keys()[0]])

In [ ]:
visualise(130)

In [ ]:
visualise(234)

In [ ]:
visualise(192)

In [ ]:
visualise(777)

In [ ]:
visualise(666)

In [ ]:
visualise(555)

In [ ]:
visualise(1222)

In [ ]:
visualise(342)

In [ ]:
visualise(872)

In [ ]:
visualise(1242)

In [ ]:
import random
randomlist = random.sample(range(0, len(df)), 200)
output = compare(list(randomlist))
# print(output)
# print(len(output))

In [ ]:
# Final metrics (Average Cosine Similarity Over SIF Embeddings)
print('Word2Vec Avg Similarity: ', sum(output[0])/len(output[0]))
print('SIF Avg Similarity: ', sum(output[1])/len(output[1]))
print('BERTwSIF Avg Similarity: ', sum(output[2])/len(output[2]))

In [ ]:
# Sample user question picked up from the internet along with context
getAnswerBert(('What do they teach at Ashoka?').lower(), ('Ashoka University is a pioneer in its focus on providing a liberal education at par with the best in the world. The aim at Ashoka is to help students become well-rounded individuals who can think critically about issues from multiple perspectives, communicate effectively and become leaders with a commitment to public service. An Ashoka education carries a strong emphasis on foundational knowledge, thorough academic research based on rigorous pedagogy, and hands-on experience with real-world challenging. The 2000-plus students on campus, drawn from 30 states and over 243 cities in India and 27 other countries, receive a world-class interdisciplinary education through undergraduate and post-graduate programmes led by internationally renowned faculty.').lower())